In [23]:
import sys
import os
import math
from tqdm import tqdm
sys.path.append(os.path.join(os.path.expanduser('~'), 'projects'))
import j_utils.munging as mg
import seaborn as sns

ppath = os.path.join(os.path.expanduser('~'), 'projects', 'lendingclub', )
dpath = os.path.join(ppath,'data')
pd.options.display.max_columns = 999

In [24]:
eval_loan_info = pd.read_feather(os.path.join(dpath,'eval_loan_info.fth'))
scaled_pmt_hist = pd.read_feather(os.path.join(dpath,'scaled_pmt_hist.fth'))
base_loan_info = pd.read_feather(os.path.join(dpath,'base_loan_info.fth'))
str_loan_info = pd.read_feather(os.path.join(dpath,'str_loan_info.fth'))

In [25]:
# add in issue_d
base_loan_info['issue_d'] = eval_loan_info['issue_d']

In [26]:
# get loans that are >= .95 maturity time, maturity paid, or status is in defaulted, charged_off, paid
done_loan_ids = eval_loan_info[(eval_loan_info['maturity_time'] >= .95) |
#                                (eval_loan_info['maturity_paid'] >= .95) |
                               (eval_loan_info['loan_status'].isin(['paid', 'charged_off', 'defaulted']))
                              ]['id'].values

In [27]:
eval_loan_info[(eval_loan_info['maturity_paid'] >= .5) &
               (eval_loan_info['maturity_time'] <= .5)]

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35
23958,2019-02-01,2017-06-01,1.000000,0.400000,-1.0,-1.0,0.000000,1.241017,0,0,paid,687119,0.161802,0.146369,0.131183,0.116237,0.101529,0.087053,0.072806,0.058784,0.044984,0.031400,0.018030,0.004870,-0.008084,-0.020835,-0.033386,-4.574210e-02,-0.057905,-0.069879,-0.081667,-0.093273,-0.104698,-0.115947,-0.127021,-0.137926,-0.148662,-0.159233,-0.169642,-0.179891,-0.189983,-0.199922,-0.209708
204345,2018-04-01,2017-05-01,1.000000,0.416667,-1.0,-1.0,0.000000,1.144755,0,0,paid,8356675,0.100897,0.092200,0.083586,0.075053,0.066601,0.058228,0.049934,0.041719,0.033580,0.025518,0.017532,0.009620,0.001783,-0.005981,-0.013673,-2.129259e-02,-0.028841,-0.036320,-0.043728,-0.051068,-0.058340,-0.065544,-0.072682,-0.079753,-0.086759,-0.093701,-0.100578,-0.107392,-0.114143,-0.120832,-0.127459
273374,2017-02-01,2017-01-01,1.000000,0.483333,-1.0,-1.0,0.000000,1.012884,0,0,paid,12687026,0.010340,0.009503,0.008666,0.007831,0.006998,0.006165,0.005335,0.004505,0.003677,0.002850,0.002025,0.001201,0.000378,-0.000443,-0.001263,-2.081879e-03,-0.002899,-0.003715,-0.004530,-0.005343,-0.006155,-0.006966,-0.007775,-0.008583,-0.009390,-0.010195,-0.010999,-0.011802,-0.012603,-0.013403,-0.014202
307929,2018-08-01,2017-02-01,1.000000,0.466667,-1.0,-1.0,0.000000,1.200965,0,0,paid,15620171,0.130680,0.116834,0.103189,0.089743,0.076491,0.063432,0.050561,0.037877,0.025375,0.013054,0.000910,-0.011059,-0.022857,-0.034485,-0.045947,-5.724552e-02,-0.068382,-0.079360,-0.090182,-0.100850,-0.111366,-0.121733,-0.131953,-0.142029,-0.151962,-0.161755,-0.171411,-0.180930,-0.190316,-0.199570,-0.208694
340405,2018-08-01,2017-03-01,1.000000,0.450000,-1.0,-1.0,0.000000,1.156296,0,0,paid,19676984,0.093989,0.081745,0.069662,0.057738,0.045971,0.034358,0.022898,0.011588,0.000425,-0.010591,-0.021464,-0.032195,-0.042787,-0.053241,-0.063559,-7.374297e-02,-0.083795,-0.093718,-0.103512,-0.113179,-0.122722,-0.132142,-0.141441,-0.150621,-0.159683,-0.168629,-0.177460,-0.186178,-0.194785,-0.203283,-0.211672
376790,2019-01-01,2017-03-01,1.000000,0.450000,-1.0,-1.0,0.000000,1.205780,0,0,paid,24284394,0.128374,0.113331,0.098528,0.083960,0.069623,0.055514,0.041628,0.027962,0.014511,0.001273,-0.011757,-0.024581,-0.037205,-0.049630,-0.061861,-7.390122e-02,-0.085753,-0.097420,-0.108906,-0.120213,-0.131344,-0.142303,-0.153093,-0.163715,-0.174174,-0.184472,-0.194611,-0.204595,-0.214426,-0.224106,-0.233638
396844,2018-02-01,2016-12-01,0.983357,0.500000,1.0,1.0,427.622772,1.148491,1,1,charged_off,27730725,0.113435,0.106729,0.100072,0.093463,0.086902,0.080388,0.073921,0.067501,0.061127,0.054799,0.048517,0.042279,0.036087,0.029939,0.023835,1.777438e-02,0.011757,0.005783,-0.000148,-0.006037,-0.011884,-0.017689,-0.023453,-0.029177,-0.034859,-0.040501,-0.046104,-0.051666,-0.057190,-0.062674,-0.068119
406474,2019-06-01,2017-03-01,0.524953,0.450000,-1.0,-1.0,20540.064453,0.756597,0,0,current,29023393,-0.279193,-0.286230,-0.293161,-0.299988,-0.306712,-0.313337,-0.319862,-0.326291,-0.332624,-0.338863,-0.345011,-0.351068,-0.357036,-0.362918,-0.368713,-3.744241e-01,-0.380052,-0.385599,-0.391066,-0.396455,-0.401766,-0.407001,-0.412162,-0.417249,-0.422264,-0.427209,-0.432083,-0.436890,-0.441628,-0.446301,-0.450908
414181,2016-12-01,2016-12-01,1.000000,0.500000,-1.0,-1.0,0.000000,1.001799,0,0,paid,30115458,-0.001252,-0.002080,-0.002907,-0.003732,-0.004556,-0.005379,-0.006200,-0.007020,-0.007839,-0.008656,-0.009472,-0.010287,-0.011100,-0.011912,-0.012722,-1.353148e-02,-0.014339,-0.015146,-0.015951,-0.016755,-0.017558,-0.018359,-0.019159,-0.019958,-0.020755,-0.021552,-0.022346,-0.023140,-0.023932,-0.024723,-0.025513
421983,2017-02-01,2017-01-01,1.000000,0.483333,-1.0,-1.0,0.000000,1.005996,0,0,paid,31266963,0.003314,0.

In [18]:
ex = eval_loan_info[~eval_loan_info['loan_status'].isin(['paid', 'current', 'grace_15'])]

In [21]:
ex = ex[ex['target_strict'] != 1]
ex = ex.iloc[np.r_[0:4, -4:0]]

In [22]:
ex

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35
267396,2019-06-01,2014-03-01,0.997880,1.000000,0.970909,0.999103,57.751637,1.635008,0,0.0,late_30,12458945,0.444150,0.409697,0.376375,0.344141,0.312955,0.282776,0.253568,0.225294,0.197920,0.171412,0.145739,0.120869,0.096773,0.073424,0.050794,0.028856,0.007587,-0.013038,-0.033043,-0.052448,-0.071276,-0.089547,-0.107281,-0.124495,-0.141209,-0.157441,-0.173206,-0.188520,-0.203400,-0.217860,-0.231915
287554,2019-06-01,2014-04-01,0.983371,1.000000,1.000000,1.000000,353.217316,1.392518,1,0.0,charged_off,13477309,0.232296,0.203329,0.175299,0.148172,0.121913,0.096492,0.071876,0.048037,0.024946,0.002576,-0.019100,-0.040107,-0.060468,-0.080207,-0.099346,-0.117907,-0.135911,-0.153376,-0.170322,-0.186767,-0.202729,-0.218225,-0.233270,-0.247881,-0.262072,-0.275858,-0.289253,-0.302270,-0.314922,-0.327221,-0.339179
291285,2019-06-01,2014-05-01,0.982974,1.000000,0.970000,0.992783,345.350159,1.579284,0,0.0,late_30,13998262,0.397573,0.364721,0.332931,0.302166,0.272386,0.243554,0.215637,0.188601,0.162413,0.137042,0.112459,0.088635,0.065543,0.043157,0.021450,0.000400,-0.020018,-0.039826,-0.059044,-0.077695,-0.095798,-0.113372,-0.130435,-0.147006,-0.163100,-0.178736,-0.193927,-0.208690,-0.223038,-0.236987,-0.250549
296903,2019-06-01,2014-04-01,0.983274,1.000000,0.984444,0.998028,460.823639,1.545817,0,0.0,late_120,14579287,0.367956,0.335800,0.304685,0.274571,0.245422,0.217202,0.189876,0.163413,0.137780,0.112947,0.088884,0.065565,0.042963,0.021051,-0.000196,-0.020800,-0.040785,-0.060173,-0.078985,-0.097240,-0.114959,-0.132161,-0.148863,-0.165082,-0.180836,-0.196139,-0.211009,-0.225459,-0.239503,-0.253156,-0.266431
2374729,2019-06-01,2019-03-01,0.000000,0.083333,0.736667,0.000000,24081.931641,0.000000,0,0.0,late_120,149405288,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2374746,2019-06-01,2019-03-01,0.000000,0.083333,0.736667,0.000000,22115.998047,0.000000,0,0.0,late_120,149406489,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2374971,2019-06-01,2019-03-01,0.000000,0.083333,1.000000,1.000000,4810.364746,0.000000,1,0.0,charged_off,149421188,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2375679,2019-06-01,2019-03-01,0.000000,0.083333,0.736667,0.000000,6689.418457,0.000000,0,0.0,late_120,149469048,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


In [ ]:
'''
things that I always expect to be true about eval_loan_info
1) if loan_status is not paid,current,grace_15, then target_strict must be 1
2) if loan_status isn't current, stat_adj values should be higher than non_stat adj values
3) if loan_status is a bad status, maturity_time_stat_adj and maturity_paid_stat_adj should be higher than
unadjusted version
3) 
'''